# Persistindo modelos de machine learning

<img src='https://www.oreilly.com/library/view/head-first-python/9781449397524/httpatomoreillycomsourceoreillyimages1368712.png.jpg'/>

## Introdução

Uma vez que finalizamos as análises nos dados e a etapa de experimentação com técnicas de aprendizagem de máquinas, podemos persistir nossa melhor solução para viabilizar seu uso. Nesta aula, utilizaremos mecanismos do próprio Python para serializar o modelo de identificação de sobreviventes do Titanic, que fizemos na última aula.

### O que vamos fazer?

- Melhorar nosso pré-processamento de dados;
- Estabelecer pipelines;
- Salvar nosso modelo criado e embarcá-lo numa aplicação WEB Flask.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Iremos continuar usando a base pública [Titanic](https://www.kaggle.com/c/titanic/data), que já está no nosso repositório (`titanic.csv`) e aplicar as principais limpezas que utilizamos na última aula.

In [ ]:
df_titanic = pd.read_csv("bases/titanic.csv")
df_titanic.head()

In [ ]:
# -- Vamos excluir algumas colunas e também as linhas que não possuem valor de alvo (Survived)

df_titanic.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin'], axis=1, inplace=True)
df_titanic.dropna(subset=['Survived'], axis=0, inplace=True)
df_titanic.head()

In [ ]:
df_titanic.shape

In [ ]:
df_titanic.isnull().sum()

### Pipelines

<img src='http://frankchen.xyz/images/15231783974167.jpg'/>

Nosso *pipeline* deverá tratar dados ausentes, e numéricos e categóricos. Dessa forma, dependendo do tipo da coluna, um tipo de pré-processamento será feito.

Para isso, inicialmente, vamos identificar e isolar os atributos do nosso DataFrame por tipo (categóricos e numéricos), porém, antes iremos deixar bossos atributos de treino ($X$) e o alvo ($y$) de forma explícita.

In [ ]:
X = df_titanic.drop(["Survived"], axis=1)
y = df_titanic["Survived"]

In [ ]:
X.dtypes

In [ ]:
cat_attrs = X.select_dtypes(['object']).columns
cat_attrs

In [ ]:
num_attrs = X.select_dtypes(['int', 'float']).columns
num_attrs

Muito bem, agora, iremos criar ''subpipelines'' com o seguinte fluxo:

- Dados categóricos:
 - Tratar valores ausentes
   - Iremos atribuir o valor mais comum presente no conjunto de treino
 - Codificar valores usando o `OrdinalEncoder`
   - Dessa forma, teremos valores inteiros e compatíveis com o Scikit-Learn
 

- Dados numéricos
 - Tratar valores ausentes
   - Podemos tratar utilizando a média do conjunto de treino
 - Padronizar valores usando o `StandardScaler`
   - Dados numéricos podem estar em escalas de valores diferentes. Uma das práticas da estatística é a normalização dos dados com a centralização da média em zero e escala em desvio padrão ($z = \frac{(X_i - \mu)}{\sigma}$). Esse processo gera algo muito próximo a uma distribuição normal. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler


cat_transformers = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ord_encoder', OrdinalEncoder())
])

num_transformers = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('standardizer', StandardScaler())
])

Em seguida, iremos utilizar o `ColumnTransformer` para aplicar as transformações nas colunas corretas dos nossos conjuntos de dados. 

Lembre-se que já identificamos quais são as colunascategóricas e numéricas, através do método `select_dtypes()`. Elas foram armazenadas nas variáveis `cat_attrs` e `num_attrs`, respectivamente.

In [ ]:
from sklearn.compose import ColumnTransformer


preprocessor = ColumnTransformer(transformers=[
    ('cat', cat_transformers, cat_attrs),
    ('num', num_transformers, num_attrs),
])

In [ ]:
## APENAS PARA VISUALIZAÇÃO!!

X_ = pd.DataFrame(preprocessor.fit_transform(X), 
                  columns=cat_attrs.to_list() + num_attrs.to_list())

X_.head()

Show! Agora que já temos nossa etapa de pré-processamento pronta, vamos montar nosso pipeline 'principal' que irá recepcionar os dados, executar `preprocessor` e, então aplicar nosso modelo.

Continuarei usando redes neurais, com os mesmos parâmetros vistos na última aula, porém sinta-se livre para usar qualquer modelo.

In [ ]:
from sklearn.neural_network import MLPClassifier


model = MLPClassifier(hidden_layer_sizes=(15, 90), activation='relu', solver='adam', 
                      max_iter=300, random_state=11)

In [ ]:
clf = Pipeline(steps=[
    ('Pré-processamento', preprocessor),
    ('Classificação', model)
])

Ah, não podemos esquecer que precisaremos ter conjuntos de treino e teste (com seus respectivos alvos separados): 

- Treino: $X\_train$, $y\_train$
- Teste: $X\_test$, $y\_test$

Para nos ajudar a criar esses subconjuntos, usaremos o método [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) novamente:

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33)

TUDO PRONTINHO para usarmos nosso Pipeline `clf`... vamos testar?

In [ ]:
clf.fit(X_train, y_train)

Vamos verificar sua performance com a `Scikit-plot`?

In [ ]:
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

In [ ]:
import scikitplot as skplt


fig, ax = plt.subplots(1, 2, figsize=(15, 5))

skplt.metrics.plot_confusion_matrix(y_test, y_pred, normalize=True, ax=ax[0])
skplt.metrics.plot_roc(y_test, y_proba, ax=ax[1])

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score


print(classification_report(y_pred, y_test))
print("Acurácia: %.4f" % accuracy_score(y_pred, y_test))
print("AUC: %.4f" % roc_auc_score(y_pred, y_test))

## Salvando meu modelo

<img src='https://www.smartfile.com/blog/wp-content/uploads/2015/11/python-pickle-800x200.png'/>

O módulo `pickle`, do Python, é usado para **serializar** e **desserializar** uma estrutura de objetos Python. <mark>Qualquer objeto no Python pode ser modificado para que possa ser salvo no disco.</mark> 

O que o pickle faz é que ele “serializa” o objeto antes de gravá-lo no arquivo. Esse processo consiste de formas de de converter um objeto python (lista, ditado etc.) em um fluxo de caracteres que contenham todas as informações necessárias para reconstruir o objeto em outro script python.

- No caso específico de objetos do scikit-learn, podemos utilizar o módulo `joblib`, que é mais eficiente ao lidar com grandes matrizes numpy internamente ([mais detalhes >>](https://scikit-learn.org/stable/modules/model_persistence.html))

In [ ]:
import joblib


joblib.dump(clf, './meu_modelo.pkl')

In [ ]:
clf_disk = joblib.load('meu_modelo.pkl')

In [ ]:
X_test.head()

In [ ]:
clf_disk.predict(X_test)[:3]

In [ ]:
y_test[:3]

In [ ]:
instancia = pd.DataFrame([{
    'Pclass': 2,
    'Sex': 'male',
    'Age': 21,
    'SibSp': 0,
    'Parch': 0,
    'Embarked': 'S'
}])

instancia

In [ ]:
inst_pred = clf_disk.predict(instancia)
inst_pred

In [ ]:
inst_proba = clf_disk.predict_proba(instancia)
inst_proba

In [ ]:
plt.pie(inst_proba, explode=(0.15, 0),  autopct="%.2f%%",
        labels=['Prob. Morrer', 'Prob. Sobreviver'])

### Vamos embarcar isso numa webapp?